# Predicción de Expansión de Cobertura 5G en Colombia (2025-2028)

Modelo basado en patrones históricos de adopción de tecnologías móviles (2G→3G→4G→5G).

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

## Carga y Análisis de Patrones Históricos

In [2]:
df = pd.read_csv('data/cobertura_movil_con_datos_geograficos.csv', low_memory=False)

df_agg = df.groupby(['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'PROVEEDOR']).agg({
    'COBERTURA 2G': lambda x: (x == 'S').sum(),
    'COBERTURA 3G': lambda x: (x == 'S').sum(),
    'COBERTURA HSPA+, HSPA+DC': lambda x: (x == 'S').sum(),
    'COBERTURA 4G': lambda x: (x == 'S').sum(),
    'CENTRO POBLADO': 'count'
}).reset_index()

df_agg.columns = ['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'PROVEEDOR', 'Centros_2G', 'Centros_3G', 'Centros_HSPA', 'Centros_4G', 'Total_Centros']

print(f"Datos cargados: {df_agg.shape}")
print(f"Años disponibles: {sorted(df_agg['AÑO'].unique())}")

Datos cargados: (5708, 9)
Años disponibles: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


In [3]:
df_tech = df_agg.groupby('AÑO')[['Centros_2G', 'Centros_3G', 'Centros_HSPA', 'Centros_4G']].sum().reset_index()

fig = go.Figure()

tecnologias = {
    'Centros_2G': '2G',
    'Centros_3G': '3G', 
    'Centros_HSPA': 'HSPA+',
    'Centros_4G': '4G',
}

for col, name in tecnologias.items():
    fig.add_trace(go.Scatter(
        x=df_tech['AÑO'],
        y=df_tech[col],
        mode='lines+markers',
        name=name,
        line=dict(width=3),
        marker=dict(size=8)
    ))

fig.update_layout(
    title='Evolución Histórica de Tecnologías Móviles en Colombia',
    xaxis_title='Año',
    yaxis_title='Centros Poblados con Cobertura',
    height=600,
    hovermode='x unified'
)

fig.write_html('graficos/evolucion_tecnologias_historica.html')
fig.show()

print("\nEstadísticas por tecnología:")
print(df_tech)


Estadísticas por tecnología:
    AÑO  Centros_2G  Centros_3G  Centros_HSPA  Centros_4G
0  2015        6935        5190          2810         745
1  2016       27760       21236         13158        3523
2  2017       24443       18504         12311        4337
3  2018       21420       16246         10350        6850
4  2019       21812       17512         11975       10527
5  2020       25351       21648         15193       14657
6  2021       35221       36114         31104       23606
7  2022       33912       37522         34941       31266
8  2023       20377       25547         23745       24481


## Análisis de Curvas de Adopción

Cada tecnología sigue un patrón de curva S (sigmoid):
1. **Introducción**: Crecimiento lento inicial
2. **Crecimiento**: Aceleración exponencial
3. **Madurez**: Saturación y estabilización
4. **Declive**: Sustitución por nueva tecnología

In [4]:
def calcular_años_desde_inicio(df_tech, tecnologia):
    df_tech_copy = df_tech.copy()
    valores = df_tech_copy[tecnologia].values
    
    primer_año_activo = None
    for i, (año, val) in enumerate(zip(df_tech_copy['AÑO'], valores)):
        if val > 0:
            primer_año_activo = año
            break
    
    if primer_año_activo is None:
        return pd.Series([None] * len(df_tech_copy))
    
    años_desde_inicio = df_tech_copy['AÑO'] - primer_año_activo
    return años_desde_inicio

df_tech['Años_2G'] = calcular_años_desde_inicio(df_tech, 'Centros_2G')
df_tech['Años_3G'] = calcular_años_desde_inicio(df_tech, 'Centros_3G')
df_tech['Años_4G'] = calcular_años_desde_inicio(df_tech, 'Centros_4G')

print("Años desde inicio de cada tecnología:")
print(df_tech[['AÑO', 'Años_2G', 'Años_3G', 'Años_4G']])

Años desde inicio de cada tecnología:
    AÑO  Años_2G  Años_3G  Años_4G
0  2015        0        0        0
1  2016        1        1        1
2  2017        2        2        2
3  2018        3        3        3
4  2019        4        4        4
5  2020        5        5        5
6  2021        6        6        6
7  2022        7        7        7
8  2023        8        8        8


## Modelo de Predicción 5G

Estrategia: Usar el patrón de crecimiento de 4G/LTE (tecnologías más recientes) como base para predecir 5G.

In [5]:
df_4g_growth = df_tech[df_tech['Centros_4G'] > 0].copy()
df_4g_growth['Años_desde_inicio'] = range(len(df_4g_growth))

años_historicos = df_4g_growth['Años_desde_inicio'].values
valores_4g = df_4g_growth['Centros_4G'].values

X_4g = años_historicos.reshape(-1, 1)
y_4g = valores_4g

modelo_4g = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
modelo_4g.fit(X_4g, y_4g)

print("Patrón de crecimiento 4G capturado")
print(f"R² del modelo base: {modelo_4g.score(X_4g, y_4g):.4f}")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=años_historicos,
    y=valores_4g,
    mode='markers',
    name='4G Real',
    marker=dict(size=10, color='blue')
))

años_fit = np.linspace(0, max(años_historicos), 100)
valores_fit = modelo_4g.predict(años_fit.reshape(-1, 1))

fig.add_trace(go.Scatter(
    x=años_fit,
    y=valores_fit,
    mode='lines',
    name='Curva Ajustada',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Curva de Adopción 4G - Modelo Base para 5G',
    xaxis_title='Años desde inicio',
    yaxis_title='Centros con Cobertura',
    height=500
)

fig.write_html('graficos/curva_adopcion_4g.html')
fig.show()

Patrón de crecimiento 4G capturado
R² del modelo base: 1.0000


In [6]:
año_inicio_5g = 2025
años_prediccion = [2025, 2026, 2027, 2028]
años_desde_inicio_5g = [0, 1, 2, 3]

factor_aceleracion = 1.3

predicciones_5g_base = modelo_4g.predict(np.array(años_desde_inicio_5g).reshape(-1, 1))
predicciones_5g_ajustadas = predicciones_5g_base * factor_aceleracion

df_pred_5g = pd.DataFrame({
    'AÑO': años_prediccion,
    'Años_desde_inicio': años_desde_inicio_5g,
    'Centros_5G_predicho': predicciones_5g_ajustadas
})

df_pred_5g['Centros_5G_predicho'] = df_pred_5g['Centros_5G_predicho'].clip(lower=0).astype(int)

print("\nPredicciones de Expansión 5G:")
print(df_pred_5g)

print("\nResumen de proyecciones:")
for _, row in df_pred_5g.iterrows():
    print(f"  {int(row['AÑO'])}: {int(row['Centros_5G_predicho']):,} centros con 5G")


Predicciones de Expansión 5G:
    AÑO  Años_desde_inicio  Centros_5G_predicho
0  2025                  0                  968
1  2026                  1                 4580
2  2027                  2                 5638
3  2028                  3                 8905

Resumen de proyecciones:
  2025: 968 centros con 5G
  2026: 4,580 centros con 5G
  2027: 5,638 centros con 5G
  2028: 8,905 centros con 5G


In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_tech['AÑO'],
    y=df_tech['Centros_4G'],
    mode='lines+markers',
    name='4G (Real)',
    line=dict(color='blue', width=3),
    marker=dict(size=10)
))

fig.add_trace(go.Scatter(
    x=df_pred_5g['AÑO'],
    y=df_pred_5g['Centros_5G_predicho'],
    mode='lines+markers',
    name='5G (Predicción)',
    line=dict(color='red', width=3, dash='dash'),
    marker=dict(size=10, symbol='star')
))

fig.update_layout(
    title='Proyección de Expansión 5G en Colombia (2025-2028)',
    xaxis_title='Año',
    yaxis_title='Centros Poblados con Cobertura',
    height=600,
    hovermode='x unified'
)

fig.write_html('graficos/prediccion_5g_nacional.html')
fig.show()

## Predicciones por Departamento

In [23]:
top_depts = df_agg.groupby('DEPARTAMENTO')['Total_Centros'].sum().nlargest(8).index

df_dept_4g = df_agg[df_agg['AÑO'] >= 2021].groupby(['DEPARTAMENTO', 'AÑO'])['Centros_4G'].sum().reset_index()

predicciones_dept = []

for dept in top_depts:
    df_dept = df_dept_4g[df_dept_4g['DEPARTAMENTO'] == dept].copy()
    
    if len(df_dept) < 2:
        continue
    
    total_4g_dept = df_dept['Centros_4G'].sum()
    proporcion_dept = total_4g_dept / df_tech[df_tech['AÑO'] >= 2021]['Centros_4G'].sum()
    
    for _, row_pred in df_pred_5g.iterrows():
        predicciones_dept.append({
            'DEPARTAMENTO': dept,
            'AÑO': int(row_pred['AÑO']),
            'Centros_5G_predicho': int(row_pred['Centros_5G_predicho'] * proporcion_dept)
        })

df_pred_5g_dept = pd.DataFrame(predicciones_dept)


df_pred_5g_dept.to_csv('predicciones_5g_dept.csv', index=False)
print("\nPredicciones 5G por Departamento (Top 8):")
print(df_pred_5g_dept.pivot(index='DEPARTAMENTO', columns='AÑO', values='Centros_5G_predicho'))


Predicciones 5G por Departamento (Top 8):
AÑO              2025  2026  2027  2028
DEPARTAMENTO                           
ANTIOQUIA         111   525   646  1021
BOLÍVAR            40   192   236   373
BOYACÁ             67   321   395   624
CAUCA              27   129   158   250
CUNDINAMARCA      125   593   730  1153
CÓRDOBA            59   282   348   549
NARIÑO             33   157   193   305
VALLE DEL CAUCA    44   211   259   410


In [9]:
fig = go.Figure()

for dept in top_depts:
    df_hist_dept = df_dept_4g[df_dept_4g['DEPARTAMENTO'] == dept]
    df_pred_dept = df_pred_5g_dept[df_pred_5g_dept['DEPARTAMENTO'] == dept]
    
    fig.add_trace(go.Scatter(
        x=df_hist_dept['AÑO'],
        y=df_hist_dept['Centros_4G'],
        mode='lines+markers',
        name=f'{dept} (4G)',
        line=dict(width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=df_pred_dept['AÑO'],
        y=df_pred_dept['Centros_5G_predicho'],
        mode='lines+markers',
        name=f'{dept} (5G pred)',
        line=dict(width=2, dash='dash')
    ))

fig.update_layout(
    title='Proyección 5G por Departamento - Top 8',
    xaxis_title='Año',
    yaxis_title='Centros con Cobertura',
    height=700,
    hovermode='x unified'
)

fig.write_html('graficos/prediccion_5g_departamentos.html')
fig.show()

## Comparación de Velocidades de Adopción

In [10]:
techs_comp = pd.DataFrame({
    'Tecnología': ['3G', '4G', '5G (Proyección)'],
    'Año_1': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[0]['Centros_3G'],
        df_tech[df_tech['Centros_4G'] > 0].iloc[0]['Centros_4G'],
        df_pred_5g.iloc[0]['Centros_5G_predicho']
    ],
    'Año_2': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[1]['Centros_3G'] if len(df_tech[df_tech['Centros_3G'] > 0]) > 1 else None,
        df_tech[df_tech['Centros_4G'] > 0].iloc[1]['Centros_4G'] if len(df_tech[df_tech['Centros_4G'] > 0]) > 1 else None,
        df_pred_5g.iloc[1]['Centros_5G_predicho']
    ],
    'Año_3': [
        df_tech[df_tech['Centros_3G'] > 0].iloc[2]['Centros_3G'] if len(df_tech[df_tech['Centros_3G'] > 0]) > 2 else None,
        df_tech[df_tech['Centros_4G'] > 0].iloc[2]['Centros_4G'] if len(df_tech[df_tech['Centros_4G'] > 0]) > 2 else None,
        df_pred_5g.iloc[2]['Centros_5G_predicho']
    ]
})

fig = go.Figure()

for tech in techs_comp['Tecnología']:
    row = techs_comp[techs_comp['Tecnología'] == tech].iloc[0]
    valores = [row['Año_1'], row['Año_2'], row['Año_3']]
    
    fig.add_trace(go.Scatter(
        x=[1, 2, 3],
        y=valores,
        mode='lines+markers',
        name=tech,
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig.update_layout(
    title='Comparación de Velocidad de Adopción por Tecnología',
    xaxis_title='Años desde introducción',
    yaxis_title='Centros con Cobertura',
    height=600,
    xaxis=dict(tickmode='linear', tick0=1, dtick=1)
)

fig.write_html('graficos/comparacion_velocidad_adopcion.html')
fig.show()

print("\nComparación de adopción (primeros 3 años):")
print(techs_comp)


Comparación de adopción (primeros 3 años):
        Tecnología  Año_1  Año_2  Año_3
0               3G   5190  21236  18504
1               4G    745   3523   4337
2  5G (Proyección)    968   4580   5638


## Resumen y Conclusiones

In [11]:
print("=" * 70)
print("PROYECCIÓN DE EXPANSIÓN 5G EN COLOMBIA (2025-2028)")
print("=" * 70)

print("\nMetodología:")
print("  - Modelo basado en curva de adopción 4G")
print("  - Factor de aceleración: 1.3x (tecnologías recientes se adoptan más rápido)")
print("  - Algoritmo: Gradient Boosting Regressor")

print("\nPredicciones Nacionales:")
for _, row in df_pred_5g.iterrows():
    año = int(row['AÑO'])
    centros = int(row['Centros_5G_predicho'])
    print(f"  {año}: {centros:,} centros con 5G")

total_2028 = df_pred_5g.iloc[-1]['Centros_5G_predicho']
total_centros = df_agg['Total_Centros'].sum() / len(df_agg['AÑO'].unique())
penetracion = (total_2028 / total_centros) * 100

print(f"\nPenetración proyectada 2028: {penetracion:.1f}%")

print("\nTop 3 departamentos proyectados para 2028:")
top_2028 = df_pred_5g_dept[df_pred_5g_dept['AÑO'] == 2028].nlargest(3, 'Centros_5G_predicho')
for _, row in top_2028.iterrows():
    print(f"  {row['DEPARTAMENTO']}: {int(row['Centros_5G_predicho']):,} centros")

crecimiento = df_pred_5g['Centros_5G_predicho'].pct_change() * 100
crecimiento_promedio = crecimiento[1:].mean()

print(f"\nCrecimiento anual promedio proyectado: {crecimiento_promedio:.1f}%")
print("=" * 70)

PROYECCIÓN DE EXPANSIÓN 5G EN COLOMBIA (2025-2028)

Metodología:
  - Modelo basado en curva de adopción 4G
  - Factor de aceleración: 1.3x (tecnologías recientes se adoptan más rápido)
  - Algoritmo: Gradient Boosting Regressor

Predicciones Nacionales:
  2025: 968 centros con 5G
  2026: 4,580 centros con 5G
  2027: 5,638 centros con 5G
  2028: 8,905 centros con 5G

Penetración proyectada 2028: 19.7%

Top 3 departamentos proyectados para 2028:
  CUNDINAMARCA: 1,153 centros
  ANTIOQUIA: 1,021 centros
  BOYACÁ: 624 centros

Crecimiento anual promedio proyectado: 151.4%


## Predicciones por Municipio (Town) y Estimación de Antenas 5G

Extendemos el modelo para predecir la expansión 5G a nivel de municipio y estimar el número de antenas que se construirán por ciudad.


In [12]:
# Agregación de datos históricos por municipio
df_agg_municipio = df.groupby(['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'MUNICIPIO', 'PROVEEDOR']).agg({
    'COBERTURA 2G': lambda x: (x == 'S').sum(),
    'COBERTURA 3G': lambda x: (x == 'S').sum(),
    'COBERTURA HSPA+, HSPA+DC': lambda x: (x == 'S').sum(),
    'COBERTURA 4G': lambda x: (x == 'S').sum(),
    'CENTRO POBLADO': 'count'
}).reset_index()

df_agg_municipio.columns = ['AÑO', 'TRIMESTRE', 'DEPARTAMENTO', 'MUNICIPIO', 'PROVEEDOR', 
                             'Centros_2G', 'Centros_3G', 'Centros_HSPA', 'Centros_4G', 'Total_Centros']

# Agregación anual por municipio (sumando todos los trimestres y proveedores)
df_municipio_anual = df_agg_municipio.groupby(['AÑO', 'DEPARTAMENTO', 'MUNICIPIO']).agg({
    'Centros_4G': 'sum',
    'Total_Centros': 'sum'
}).reset_index()

print(f"Datos agregados por municipio: {df_municipio_anual.shape}")
print(f"Municipios únicos: {df_municipio_anual['MUNICIPIO'].nunique()}")
print(f"Años disponibles: {sorted(df_municipio_anual['AÑO'].unique())}")

# Filtrar municipios con datos históricos de 4G (al menos 2 años de datos)
municipios_con_historia = df_municipio_anual[df_municipio_anual['Centros_4G'] > 0].groupby('MUNICIPIO')['AÑO'].count()
municipios_validos = municipios_con_historia[municipios_con_historia >= 2].index

print(f"\nMunicipios con al menos 2 años de datos históricos de 4G: {len(municipios_validos)}")


Datos agregados por municipio: (10095, 5)
Municipios únicos: 1037
Años disponibles: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]

Municipios con al menos 2 años de datos históricos de 4G: 1027


In [13]:
# Preparar datos históricos de 4G por municipio para el modelo
df_municipio_4g = df_municipio_anual[df_municipio_anual['MUNICIPIO'].isin(municipios_validos)].copy()
df_municipio_4g = df_municipio_4g[df_municipio_4g['Centros_4G'] > 0].copy()

# Para cada municipio, calcular años desde inicio de 4G
def calcular_años_desde_inicio_municipio(df_municipio, municipio_nombre):
    df_mun = df_municipio[df_municipio['MUNICIPIO'] == municipio_nombre].copy().sort_values('AÑO')
    if len(df_mun) == 0:
        return None
    
    primer_año_4g = df_mun.iloc[0]['AÑO']
    df_mun['Años_desde_inicio'] = df_mun['AÑO'] - primer_año_4g
    return df_mun

# Aplicar a todos los municipios
lista_municipios_datos = []
for municipio in municipios_validos:
    df_mun = calcular_años_desde_inicio_municipio(df_municipio_4g, municipio)
    if df_mun is not None and len(df_mun) >= 2:
        lista_municipios_datos.append(df_mun)

df_municipios_con_tiempo = pd.concat(lista_municipios_datos, ignore_index=True)

print(f"Municipios preparados para modelado: {df_municipios_con_tiempo['MUNICIPIO'].nunique()}")
print(f"\nEjemplo de datos por municipio:")
print(df_municipios_con_tiempo[['DEPARTAMENTO', 'MUNICIPIO', 'AÑO', 'Años_desde_inicio', 'Centros_4G']].head(10))


Municipios preparados para modelado: 1027

Ejemplo de datos por municipio:
         DEPARTAMENTO  MUNICIPIO   AÑO  Años_desde_inicio  Centros_4G
0           ANTIOQUIA  ABEJORRAL  2018                  0           1
1           ANTIOQUIA  ABEJORRAL  2019                  1           7
2           ANTIOQUIA  ABEJORRAL  2020                  2           8
3           ANTIOQUIA  ABEJORRAL  2021                  3          12
4           ANTIOQUIA  ABEJORRAL  2022                  4          16
5           ANTIOQUIA  ABEJORRAL  2023                  5          14
6  NORTE DE SANTANDER     ABREGO  2016                  0           2
7  NORTE DE SANTANDER     ABREGO  2017                  1           8
8  NORTE DE SANTANDER     ABREGO  2018                  2           8
9  NORTE DE SANTANDER     ABREGO  2019                  3           8


In [14]:
# Construir modelos de predicción por municipio
# Usaremos el mismo enfoque: modelo basado en curva de adopción 4G con factor de aceleración

predicciones_municipio = []

for municipio in df_municipios_con_tiempo['MUNICIPIO'].unique():
    df_mun = df_municipios_con_tiempo[df_municipios_con_tiempo['MUNICIPIO'] == municipio].copy().sort_values('AÑO')
    dept = df_mun.iloc[0]['DEPARTAMENTO']
    
    if len(df_mun) < 2:
        continue
    
    # Preparar datos para el modelo
    años_desde_inicio = df_mun['Años_desde_inicio'].values
    valores_4g = df_mun['Centros_4G'].values
    
    X_mun = años_desde_inicio.reshape(-1, 1)
    y_mun = valores_4g
    
    # Entrenar modelo para este municipio
    try:
        modelo_mun = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
        modelo_mun.fit(X_mun, y_mun)
        
        # Predecir para años 2025-2028
        # Año base: último año con datos históricos
        ultimo_año_hist = df_mun['AÑO'].max()
        años_desde_base = df_mun[df_mun['AÑO'] == ultimo_año_hist]['Años_desde_inicio'].iloc[0]
        
        años_prediccion = [2025, 2026, 2027, 2028]
        # Calcular años desde inicio para cada año de predicción
        # Continuar la secuencia desde el último año histórico
        años_desde_inicio_pred = [
            años_desde_base + (año - ultimo_año_hist)
            for año in años_prediccion
        ]
        
        # Predicciones base 4G
        predicciones_base = modelo_mun.predict(np.array(años_desde_inicio_pred).reshape(-1, 1))
        
        # Aplicar factor de aceleración para 5G (1.3x)
        factor_aceleracion = 1.3
        predicciones_5g = predicciones_base * factor_aceleracion
        predicciones_5g = np.clip(predicciones_5g, 0, None)
        
        # Guardar predicciones
        for año, pred in zip(años_prediccion, predicciones_5g):
            predicciones_municipio.append({
                'DEPARTAMENTO': dept,
                'MUNICIPIO': municipio,
                'AÑO': año,
                'Centros_5G_predicho': int(pred)
            })
    except Exception as e:
        # Si falla el modelo para este municipio, usar proporción del departamento
        continue

df_pred_5g_municipio = pd.DataFrame(predicciones_municipio)

print(f"Predicciones generadas para {df_pred_5g_municipio['MUNICIPIO'].nunique()} municipios")
print(f"Total de registros de predicción: {len(df_pred_5g_municipio)}")
print("\nEjemplo de predicciones:")
print(df_pred_5g_municipio.head(15))


Predicciones generadas para 1027 municipios
Total de registros de predicción: 4108

Ejemplo de predicciones:
          DEPARTAMENTO  MUNICIPIO   AÑO  Centros_5G_predicho
0            ANTIOQUIA  ABEJORRAL  2025                   18
1            ANTIOQUIA  ABEJORRAL  2026                   18
2            ANTIOQUIA  ABEJORRAL  2027                   18
3            ANTIOQUIA  ABEJORRAL  2028                   18
4   NORTE DE SANTANDER     ABREGO  2025                   15
5   NORTE DE SANTANDER     ABREGO  2026                   15
6   NORTE DE SANTANDER     ABREGO  2027                   15
7   NORTE DE SANTANDER     ABREGO  2028                   15
8            ANTIOQUIA   ABRIAQUÍ  2025                    7
9            ANTIOQUIA   ABRIAQUÍ  2026                    7
10           ANTIOQUIA   ABRIAQUÍ  2027                    7
11           ANTIOQUIA   ABRIAQUÍ  2028                    7
12                META    ACACÍAS  2025                   49
13                META    ACACÍAS  20

In [15]:
# Estimación de antenas 5G por municipio
# Método: Estimar antenas basado en centros poblados con cobertura 5G
# Asumimos que 1 antena 5G puede cubrir aproximadamente 8-10 centros poblados en promedio
# (5G requiere más antenas que 4G debido a frecuencias más altas y menor alcance)

# Ratio conservador: 1 antena por cada 8 centros poblados con cobertura 5G
# Para áreas urbanas densas, puede ser menor (más antenas)
# Para áreas rurales, puede ser mayor (menos antenas, pero más alcance)

RATIO_ANTENAS_POR_CENTRO = 8  # 1 antena por cada 8 centros poblados cubiertos

df_pred_5g_municipio['Antenas_5G_Predicho'] = (
    df_pred_5g_municipio['Centros_5G_predicho'] / RATIO_ANTENAS_POR_CENTRO
).round().astype(int)

# Asegurar mínimo de 0 antenas (no negativas)
df_pred_5g_municipio['Antenas_5G_Predicho'] = df_pred_5g_municipio['Antenas_5G_Predicho'].clip(lower=0)

print("Estimación de antenas 5G completada")
print(f"\nRatio utilizado: 1 antena por cada {RATIO_ANTENAS_POR_CENTRO} centros poblados con cobertura 5G")
print("\nResumen de predicciones por año:")
print(df_pred_5g_municipio.groupby('AÑO').agg({
    'MUNICIPIO': 'count',
    'Centros_5G_predicho': 'sum',
    'Antenas_5G_Predicho': 'sum'
}).rename(columns={'MUNICIPIO': 'Municipios_Con_Prediccion'}))


Estimación de antenas 5G completada

Ratio utilizado: 1 antena por cada 8 centros poblados con cobertura 5G

Resumen de predicciones por año:
      Municipios_Con_Prediccion  Centros_5G_predicho  Antenas_5G_Predicho
AÑO                                                                      
2025                       1027                29118                 3584
2026                       1027                29118                 3584
2027                       1027                29118                 3584
2028                       1027                29118                 3584


In [16]:
# Guardar predicciones en CSV
df_pred_5g_municipio_sorted = df_pred_5g_municipio.sort_values(['AÑO', 'DEPARTAMENTO', 'MUNICIPIO'])
df_pred_5g_municipio_sorted.to_csv('data/predicciones_5g_por_municipio.csv', index=False, encoding='utf-8-sig')

print("=" * 80)
print("PREDICCIONES 5G POR MUNICIPIO (2025-2028)")
print("=" * 80)
print(f"\nTotal de municipios con predicciones: {df_pred_5g_municipio_sorted['MUNICIPIO'].nunique()}")
print(f"Archivo guardado: data/predicciones_5g_por_municipio.csv")

print("\nTop 10 municipios con más antenas proyectadas para 2028:")
top_2028_municipios = df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028].nlargest(10, 'Antenas_5G_Predicho')
for _, row in top_2028_municipios.iterrows():
    print(f"  {row['MUNICIPIO']} ({row['DEPARTAMENTO']}): {int(row['Antenas_5G_Predicho'])} antenas, {int(row['Centros_5G_predicho'])} centros")

print("\nResumen por año:")
resumen_anual = df_pred_5g_municipio_sorted.groupby('AÑO').agg({
    'Antenas_5G_Predicho': 'sum',
    'Centros_5G_predicho': 'sum',
    'MUNICIPIO': 'count'
}).rename(columns={'MUNICIPIO': 'Municipios'})
print(resumen_anual)
print("=" * 80)


PREDICCIONES 5G POR MUNICIPIO (2025-2028)

Total de municipios con predicciones: 1027
Archivo guardado: data/predicciones_5g_por_municipio.csv

Top 10 municipios con más antenas proyectadas para 2028:
  PEREIRA (RISARALDA): 34 antenas, 276 centros
  MONTERÍA (CÓRDOBA): 31 antenas, 249 centros
  ZIPAQUIRÁ (CUNDINAMARCA): 24 antenas, 191 centros
  SAN JOSÉ DE CÚCUTA (NORTE DE SANTANDER): 22 antenas, 175 centros
  CARTAGENA (BOLÍVAR): 21 antenas, 168 centros
  CAJICÁ (CUNDINAMARCA): 21 antenas, 167 centros
  TOCANCIPÁ (CUNDINAMARCA): 19 antenas, 152 centros
  SAHAGÚN (CÓRDOBA): 19 antenas, 152 centros
  MARÍA LA BAJA (BOLÍVAR): 18 antenas, 141 centros
  LORICA (CÓRDOBA): 17 antenas, 133 centros

Resumen por año:
      Antenas_5G_Predicho  Centros_5G_predicho  Municipios
AÑO                                                       
2025                 3584                29118        1027
2026                 3584                29118        1027
2027                 3584                2911

## Visualizaciones de Predicciones por Municipio


In [17]:
# Visualización 1: Top 20 municipios con más antenas proyectadas para 2028
top_20_municipios = df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028].nlargest(20, 'Antenas_5G_Predicho')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_20_municipios['Antenas_5G_Predicho'],
    y=top_20_municipios['MUNICIPIO'] + ' (' + top_20_municipios['DEPARTAMENTO'] + ')',
    orientation='h',
    marker=dict(color='#1f77b4', line=dict(color='rgba(0,0,0,0.5)', width=1)),
    text=top_20_municipios['Antenas_5G_Predicho'],
    textposition='outside'
))

fig.update_layout(
    title='Top 20 Municipios con Más Antenas 5G Proyectadas para 2028',
    xaxis_title='Número de Antenas 5G',
    yaxis_title='Municipio',
    height=700,
    yaxis=dict(autorange='reversed')
)

fig.write_html('graficos/top_municipios_antenas_5g_2028.html')
fig.show()


In [18]:
# Visualización 2: Evolución de antenas por año (agregado nacional)
evolucion_antenas = df_pred_5g_municipio_sorted.groupby('AÑO')['Antenas_5G_Predicho'].sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=evolucion_antenas['AÑO'],
    y=evolucion_antenas['Antenas_5G_Predicho'],
    mode='lines+markers',
    name='Total Antenas 5G',
    line=dict(color='#ff7f0e', width=4),
    marker=dict(size=12, symbol='diamond'),
    fill='tozeroy',
    fillcolor='rgba(255, 127, 14, 0.2)'
))

fig.update_layout(
    title='Evolución Proyectada de Antenas 5G en Colombia (2025-2028)',
    xaxis_title='Año',
    yaxis_title='Total de Antenas 5G',
    height=500,
    hovermode='x unified'
)

fig.write_html('graficos/evolucion_antenas_5g_nacional.html')
fig.show()

print("\nEvolución de antenas por año:")
for _, row in evolucion_antenas.iterrows():
    print(f"  {int(row['AÑO'])}: {int(row['Antenas_5G_Predicho']):,} antenas")



Evolución de antenas por año:
  2025: 3,584 antenas
  2026: 3,584 antenas
  2027: 3,584 antenas
  2028: 3,584 antenas


In [19]:
# Visualización 3: Distribución de antenas por departamento (2028)
antenas_por_dept_2028 = df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028].groupby('DEPARTAMENTO')['Antenas_5G_Predicho'].sum().reset_index()
antenas_por_dept_2028 = antenas_por_dept_2028.sort_values('Antenas_5G_Predicho', ascending=False)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=antenas_por_dept_2028['DEPARTAMENTO'],
    y=antenas_por_dept_2028['Antenas_5G_Predicho'],
    marker=dict(
        color=antenas_por_dept_2028['Antenas_5G_Predicho'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Antenas')
    ),
    text=antenas_por_dept_2028['Antenas_5G_Predicho'],
    textposition='outside'
))

fig.update_layout(
    title='Distribución de Antenas 5G Proyectadas por Departamento (2028)',
    xaxis_title='Departamento',
    yaxis_title='Número de Antenas 5G',
    height=600,
    xaxis=dict(tickangle=45)
)

fig.write_html('graficos/antenas_5g_por_departamento_2028.html')
fig.show()


In [20]:
# Visualización 4: Mapa de calor de municipios con antenas 5G proyectadas (2028)
# Preparar datos para el mapa
df_pred_2028 = df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028].copy()

# Obtener coordenadas de los municipios desde el dataset original
df_municipios_coords = df[['MUNICIPIO', 'DEPARTAMENTO', 'Latitud', 'Longitud']].drop_duplicates(subset=['MUNICIPIO', 'DEPARTAMENTO'])
df_municipios_coords = df_municipios_coords.groupby(['MUNICIPIO', 'DEPARTAMENTO']).agg({
    'Latitud': 'mean',
    'Longitud': 'mean'
}).reset_index()

# Merge con predicciones
df_pred_2028_map = df_pred_2028.merge(df_municipios_coords, on=['MUNICIPIO', 'DEPARTAMENTO'], how='left')
df_pred_2028_map = df_pred_2028_map.dropna(subset=['Latitud', 'Longitud'])

# Crear mapa
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=df_pred_2028_map['Latitud'],
    lon=df_pred_2028_map['Longitud'],
    mode='markers',
    marker=dict(
        size=df_pred_2028_map['Antenas_5G_Predicho'].apply(lambda x: max(5, min(x/5, 50))),
        color=df_pred_2028_map['Antenas_5G_Predicho'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Antenas 5G'),
        opacity=0.7
    ),
    text=df_pred_2028_map['MUNICIPIO'] + '<br>Antenas: ' + df_pred_2028_map['Antenas_5G_Predicho'].astype(str),
    hoverinfo='text'
))

fig.update_layout(
    title='Mapa de Antenas 5G Proyectadas por Municipio (2028)',
    mapbox=dict(
        style='open-street-map',
        center=dict(lat=4.5, lon=-74.0),
        zoom=5
    ),
    height=700
)

fig.write_html('graficos/mapa_antenas_5g_municipios_2028.html')
fig.show()

print(f"\nMunicipios con coordenadas para mapa: {len(df_pred_2028_map)}")



Municipios con coordenadas para mapa: 979


In [21]:
# Visualización 5: Evolución temporal de antenas para top 10 municipios
top_10_municipios_nombres = top_2028_municipios['MUNICIPIO'].head(10).tolist()

fig = go.Figure()

for municipio in top_10_municipios_nombres:
    df_mun_evol = df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['MUNICIPIO'] == municipio].copy()
    dept = df_mun_evol.iloc[0]['DEPARTAMENTO']
    
    fig.add_trace(go.Scatter(
        x=df_mun_evol['AÑO'],
        y=df_mun_evol['Antenas_5G_Predicho'],
        mode='lines+markers',
        name=f'{municipio} ({dept})',
        line=dict(width=2),
        marker=dict(size=8)
    ))

fig.update_layout(
    title='Evolución de Antenas 5G - Top 10 Municipios (2025-2028)',
    xaxis_title='Año',
    yaxis_title='Número de Antenas 5G',
    height=600,
    hovermode='x unified',
    legend=dict(orientation='v', yanchor='top', y=1, xanchor='left', x=1.02)
)

fig.write_html('graficos/evolucion_antenas_top_municipios.html')
fig.show()


In [22]:
# Visualización 6: Tabla resumen de predicciones por municipio y año
print("=" * 100)
print("RESUMEN DE PREDICCIONES 5G POR MUNICIPIO (2025-2028)")
print("=" * 100)

# Crear tabla pivot para mejor visualización
tabla_resumen = df_pred_5g_municipio_sorted.pivot_table(
    index=['DEPARTAMENTO', 'MUNICIPIO'],
    columns='AÑO',
    values='Antenas_5G_Predicho',
    aggfunc='sum',
    fill_value=0
)

print("\nTop 30 municipios con más antenas proyectadas (resumen por año):")
print(tabla_resumen.sort_values(2028, ascending=False).head(30))

# Guardar tabla resumen
tabla_resumen.to_csv('data/resumen_predicciones_antenas_5g_municipio.csv', encoding='utf-8-sig')

print("\n\nResumen estadístico:")
print(f"Total de municipios con predicciones: {df_pred_5g_municipio_sorted['MUNICIPIO'].nunique()}")
print(f"Total de antenas proyectadas para 2028: {df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028]['Antenas_5G_Predicho'].sum():,}")
print(f"Promedio de antenas por municipio en 2028: {df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028]['Antenas_5G_Predicho'].mean():.1f}")
print(f"Mediana de antenas por municipio en 2028: {df_pred_5g_municipio_sorted[df_pred_5g_municipio_sorted['AÑO'] == 2028]['Antenas_5G_Predicho'].median():.0f}")
print("=" * 100)


RESUMEN DE PREDICCIONES 5G POR MUNICIPIO (2025-2028)

Top 30 municipios con más antenas proyectadas (resumen por año):
AÑO                                    2025  2026  2027  2028
DEPARTAMENTO       MUNICIPIO                                 
RISARALDA          PEREIRA               34    34    34    34
CÓRDOBA            MONTERÍA              31    31    31    31
CUNDINAMARCA       ZIPAQUIRÁ             24    24    24    24
NORTE DE SANTANDER SAN JOSÉ DE CÚCUTA    22    22    22    22
BOLÍVAR            CARTAGENA             21    21    21    21
CUNDINAMARCA       CAJICÁ                21    21    21    21
                   TOCANCIPÁ             19    19    19    19
CÓRDOBA            SAHAGÚN               19    19    19    19
BOLÍVAR            MARÍA LA BAJA         18    18    18    18
CÓRDOBA            LORICA                17    17    17    17
VALLE DEL CAUCA    PALMIRA               17    17    17    17
NARIÑO             PASTO                 16    16    16    16
LA GUAJIRA   